In [1]:
"""
测试RAG评估


RAG构造数据的流程：
1、收集真实的问题和答案：question和answer
2、设置RAG的数据索引、召回模块和答案生成模块--》整个实验跑通
3、针对每个question和answer，获取其召回的上下文字段context和生成的答案

经过上述三个步骤，组成的评估数据格式：

{'question': 'Which private companies in the Americas are the largest GHG emitters according to the Carbon Majors database?',
 'ground_truths': ['The largest private companies in the Americas that are the largest GHG emitters according to the Carbon Majors database are ExxonMobil, Chevron, and Peabody.'],
 'answer': 'According to the Carbon Majors database, the largest private companies in the Americas that are the largest GHG emitters are:\n\n1. Chevron Corporation (United States)\n2. ExxonMobil Corporation (United States)\n3. ConocoPhillips Company (United States)\n4. BP plc (United Kingdom, but with significant operations in the Americas)\n5. Royal Dutch Shell plc (Netherlands, but with significant operations in the Americas)\n6. Peabody Energy Corporation (United States)\n7. Duke Energy Corporation (United States)\n8. TotalEnergies SE (France, but with significant operations in the Americas)\n9. BHP Group Limited (Australia, but with significant operations in the Americas)\n10. Rio Tinto Group (United Kingdom/Australia, but with significant operations in the Americas)\n\nPlease note that the rankings may change over time as new data becomes available.',
 'contexts': ['The private companies responsible for the most emissions during this period, according to the database, are from the United States: ExxonMobil, Chevron and Peabody.\nThe largest emitter amongst state-owned companies in the Americas is Mexican company Pemex, followed by Venezuelan company Petróleos de Venezuela, S.A.']}

将上述得到的数据集送入到Ragas中进行评估

"""

"""
RAG评估指标：
1、忠实度（faithfulness）
   衡量生成的答案answer与给定上下文context的事实一致性：如果答案(answer)中提出的所有基本事实(claims)都可以从给定的上下文(context)中推断出来，
   则生成的答案被认为是忠实的。
   Faithfulness score = number of claims that can be infered from given context / total number of claim in the generated answer.

   ragas论文提高答案的忠实度是利用LLM进行判断的。

2、答案相关性（Answer Revevancy）
   评估生成的答案(answer)与用户问题(question)之间相关程度。不完整或包含冗余信息的答案将获得较低分数
   该指标是通过计算question和answer获得的，它的取值范围在 0 到 1 之间，其中分数越高表示相关性越好

   ragas论文中提高的答案相关性是利用Openai对答案生成若干个question，然后利用生成的question和标准的question通过openaiembedding获取对应embedding，并计算所有得分的平均值。
   计算步骤：

       给定生成的 答案，基于该答案内容，用LLM生成n个潜在问题；
       用文本转向量模型（text-embedding-ada-002 model）将问题和潜在问题文本转为向量，计算它们的相似性：

   

3、上下文精度（Context Precision）
    上下文精确度衡量上下文中所有相关的真实信息是否被排在了较高的位置。理想情况下，所有相关的信息块都应该出现在排名的最前面。这个指标是根据问题和上下文来计算的，数值范围在0到1之间，分数越高表示精确度越好

    context precision@k = Sum(precision@k) / total number of relevant items in the top K results

    


    

4、上下文召回率（Context Recall）
     用来衡量检索到的上下文与被视为事实真相的标注答案的一致性程度。它根据事实真相和检索到的上下文来计算,数值范围在0到1之间,数值越高表示性能越好; 
     为了从事实真相的答案中估计上下文召回率,需要分析真实答案中的每个句子是否可以归因于检索到的上下文。在理想情况下,事实真相答案中的所有句子都应该能够对应到检索到的上下文中。

*5、上下文相关性（Context Relevancy）

    ragas论文：对于给定的question和context，利用LLM从context中抽取相关的句子， 抽取到的相关性句子数量 除以  上下文中全部句子数量 就是context relevancy。
    给定问题和上下文文本，用LLM从上下文文本中提取有助于回答问题的句子，然后计算得分。




RAG的评估总体分为两类：
1、检索器的评估Retriever评价（针对Embedding）
   相关指标：
   1）Context Precision: 
   2）Context Recall：

2、生成器的评价Generator  (主要针对LLM)
   1）Faithfulless：LLM根据query + Context 生成最终回答在语义上有没有遵循Context，避免回答跳出Context的范围，产生幻觉。
   2）Answer Relevance: LLM根据query + context 生成的最终回答有没有紧扣query的要求，答案是否完整或包含冗余信息。
    
"""

# HF_ENDPOINT=https://hf-mirror.com
import os
os.environ["HF_ENDPOINT"] =  "https://hf-mirror.com"

import json

In [2]:

from ragas.metrics import LLMContextRecall, Faithfulness, AnswerCorrectness, FactualCorrectness

from langchain_openai import ChatOpenAI

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from ragas import evaluate

# HuggingfaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import  HuggingFaceBgeEmbeddings

/root/autodl-tmp/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface/

model_name = "BAAI/bge-base-zh-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, 
    cache_folder="./bge-base-zh-v1.5/",
    encode_kwargs=encode_kwargs
)


In [4]:

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(api_key="EMPTY", base_url="http://localhost:8000/v1", model="Qwen2.5-7B-Instruct-AWQ"))


metrics = [LLMContextRecall(), FactualCorrectness(), Faithfulness()]

# https://docs.ragas.io/en/latest/references/embeddings/?h=embedding#ragas.embeddings.HuggingfaceEmbeddings




In [5]:
# evaluator_llm.generate_text("你是谁？")

LangchainEmbeddingsWrapper??

# 测试chatopenai是否连接成功？

# evaluator_llm.langchain_llm.invoke("你是谁？")


Init signature:
LangchainEmbeddingsWrapper(
    embeddings: 'Embeddings',
    run_config: 't.Optional[RunConfig]' = None,
)
Source:        
class LangchainEmbeddingsWrapper(BaseRagasEmbeddings):
    """
    Wrapper for any embeddings from langchain.
    """

    def __init__(
        self, embeddings: Embeddings, run_config: t.Optional[RunConfig] = None
    ):
        self.embeddings = embeddings
        if run_config is None:
            run_config = RunConfig()
        self.set_run_config(run_config)

    def embed_query(self, text: str) -> List[float]:
        """
        Embed a single query text.
        """
        return self.embeddings.embed_query(text)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        Embed multiple documents.
        """
        return self.embeddings.embed_documents(texts)

    async def aembed_query(self, text: str) -> List[float]:
        """
        Asynchronously embed a single query text.
        """
        retu

In [6]:

# 获取数据集

# 数据集的样本格式

# one_example = {'question': 'Which private companies in the Americas are the largest GHG emitters according to the Carbon Majors database?',
#  'ground_truths': ['The largest private companies in the Americas that are the largest GHG emitters according to the Carbon Majors database are ExxonMobil, Chevron, and Peabody.'],
#  'answer': 'According to the Carbon Majors database, the largest private companies in the Americas that are the largest GHG emitters are:\n\n1. Chevron Corporation (United States)\n2. ExxonMobil Corporation (United States)\n3. ConocoPhillips Company (United States)\n4. BP plc (United Kingdom, but with significant operations in the Americas)\n5. Royal Dutch Shell plc (Netherlands, but with significant operations in the Americas)\n6. Peabody Energy Corporation (United States)\n7. Duke Energy Corporation (United States)\n8. TotalEnergies SE (France, but with significant operations in the Americas)\n9. BHP Group Limited (Australia, but with significant operations in the Americas)\n10. Rio Tinto Group (United Kingdom/Australia, but with significant operations in the Americas)\n\nPlease note that the rankings may change over time as new data becomes available.',
#  'contexts': ['The private companies responsible for the most emissions during this period, according to the database, are from the United States: ExxonMobil, Chevron and Peabody.\nThe largest emitter amongst state-owned companies in the Americas is Mexican company Pemex, followed by Venezuelan company Petróleos de Venezuela, S.A.']}


with open("ragas评估数据集.json", "r") as fin:
    examples = json.load(fin)


In [7]:

from datasets import Dataset

eval_dataset = Dataset.from_list(examples)

eval_dataset[:5]



{'query': ['什么是深度学习？',
  '太阳系有多少颗行星？',
  '量子计算的主要特点是什么？',
  '中国的四大发明是什么？',
  '电子商务的优势是什么？'],
 'context': [['深度学习是机器学习的一个分支，使用神经网络进行多层信息处理。', '它常用于语音识别、图像识别和自然语言处理等领域。'],
  ['太阳系包含八颗行星，依次为水星、金星、地球、火星、木星、土星、天王星和海王星。', '它们围绕太阳运行，构成了太阳系的主要组成部分。'],
  ['量子计算利用量子比特进行计算，能够并行处理多个状态。', '其特点包括量子叠加、量子纠缠和量子干涉等。'],
  ['中国古代四大发明包括造纸术、指南针、火药和印刷术。', '这些发明对世界科技和文化发展产生了深远影响。'],
  ['电子商务能够缩短交易时间、降低成本，并实现24小时全球交易。', '它提高了消费者的购物便捷性和企业的市场扩展性。']],
 'ground_truth_answer': ['深度学习是机器学习的一个子领域，通过神经网络进行多层信息处理。',
  '太阳系中有八颗行星。',
  '量子计算的特点包括量子叠加、量子纠缠和量子干涉。',
  '中国四大发明是造纸术、指南针、火药和印刷术。',
  '电子商务可以降低交易成本，增加便捷性。'],
 'generated_answer': ['深度学习是一种机器学习方法，通过神经网络进行多层数据处理。',
  '太阳系包含八颗行星。',
  '量子计算主要特点是量子叠加和量子纠缠。',
  '四大发明包括造纸术、指南针、火药和印刷术。',
  '电子商务能降低交易成本，并带来更高的便捷性。']}

In [8]:
# from datasets import load_dataset
# dataset = load_dataset("explodinggradients/amnesty_qa", "english_v3")
# dataset

In [11]:
from IPython.display import display
from ragas.metrics import ContextPrecision, ContextRecall

metrics = [Faithfulness(), AnswerCorrectness(),ContextPrecision(), ContextRecall()]
"""

faithfulness指标评估需要数据集中包含：['user_input', 'response', 'retrieved_contexts']
"""
eval_dataset_bak = eval_dataset


eval_dataset_new = eval_dataset.rename_columns({"query": "user_input", "context": "retrieved_contexts", "generated_answer":"response", "ground_truth_answer": "reference"})


result1 = evaluate(eval_dataset_new, metrics=metrics, llm=evaluator_llm, embeddings=embeddings)

display(result1.to_pandas())


Evaluating: 100%|██████████| 80/80 [00:31<00:00,  2.52it/s]


,user_input,retrieved_contexts,response,reference,faithfulness,answer_correctness,context_precision,context_recall
0,什么是深度学习？,"[深度学习是机器学习的一个分支，使用神经网络进行多层信息处理。, 它常用于语音识别、图像识别...",深度学习是一种机器学习方法，通过神经网络进行多层数据处理。,深度学习是机器学习的一个子领域，通过神经网络进行多层信息处理。,1.0,0.876493,1.0,1.0
1,太阳系有多少颗行星？,"[太阳系包含八颗行星，依次为水星、金星、地球、火星、木星、土星、天王星和海王星。, 它们围绕...",太阳系包含八颗行星。,太阳系中有八颗行星。,1.0,0.997420,1.0,1.0
2,量子计算的主要特点是什么？,"[量子计算利用量子比特进行计算，能够并行处理多个状态。, 其特点包括量子叠加、量子纠缠和量子...",量子计算主要特点是量子叠加和量子纠缠。,量子计算的特点包括量子叠加、量子纠缠和量子干涉。,1.0,0.835563,1.0,1.0
3,中国的四大发明是什么？,"[中国古代四大发明包括造纸术、指南针、火药和印刷术。, 这些发明对世界科技和文化发展产生了深...",四大发明包括造纸术、指南针、火药和印刷术。,中国四大发明是造纸术、指南针、火药和印刷术。,1.0,0.980668,1.0,1.0
4,电子商务的优势是什么？,"[电子商务能够缩短交易时间、降低成本，并实现24小时全球交易。, 它提高了消费者的购物便捷性...",电子商务能降低交易成本，并带来更高的便捷性。,电子商务可以降低交易成本，增加便捷性。,1.0,0.994829,1.0,1.0
5,世界上最长的河流是哪一条？,"[世界上最长的河流是尼罗河，全长约6650公里。, 尼罗河流经非洲东北部多个国家，最终流入地...",尼罗河是世界上最长的河流。,世界上最长的河流是尼罗河。,1.0,0.991609,1.0,1.0
6,什么是区块链技术？,"[区块链是一种分布式账本技术，具有去中心化、安全透明等特点。, 它常用于加密货币交易，如比特币。]",区块链是一种去中心化的分布式账本技术。,区块链是一种分布式账本技术，具有去中心化和安全透明的特点。,1.0,0.482777,1.0,0.5
7,全球变暖的主要原因是什么？,"[全球变暖的主要原因是温室气体增加，尤其是二氧化碳的排放。, 工业活动、化石燃料燃烧是温室气...",温室气体增加是全球变暖的主要原因。,全球变暖主要由于温室气体增加，尤其是二氧化碳排放。,1.0,0.727080,1.0,1.0
8,计算机的主要组成部分有哪些？,"[计算机主要由中央处理器（CPU）、内存、存储设备和输入输出设备组成。, 这些部件协同工作，...",计算机的组成包括中央处理器、内存和输入输出设备。,计算机的组成包括CPU、内存、存储设备和输入输出设备。,1.0,0.736475,1.0,1.0
9,光速是多少？,"[光速在真空中为每秒299,792,458米。, 这是物理学中一个基本常数，表示光在真空中的...","光速约为299,792,458米每秒。","光速在真空中约为299,792,458米每秒。",1.0,0.979919,1.0,1.0


In [ ]:
evaluate(eval_dataset, metrics=metrics, llm=evaluator_llm, embeddings=embeddings)



